<a href="https://colab.research.google.com/github/SahilCarterr/FinetuningLLM/blob/main/mistral_7b_v3_finetuned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 1. Installing Libraries

In [1]:
!pip -q install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip -q install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 98.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the so

#### 2. Loading the model and tokenizer

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-v0.3",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Mistral patching release 2024.7
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/137k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Unsloth: Will load unsloth/mistral-7b-v0.3-bnb-4bit as a legacy tokenizer.


##### 2.1 Loading Peft Config

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


##### 2.2 Formatting prompt and initializing `alpaca_prompt`

In [4]:
alpaca_prompt = """Below is an question which corresponds to topic. Write a topic that appropriate for the question.

### question:
{}

### topic:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    question = examples["question"]
    topic = examples["topic"]
    texts = []
    for question, topic in zip(question, topic):

        text = alpaca_prompt.format(question, topic) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

#### 3. Preprocessing the data

In [5]:
import pandas as pd
df = pd.read_csv('/content/data_withoutLENE.csv')
unique_topics = df['topic'].unique()
topic_mapping = {topic: i for i, topic in enumerate(unique_topics)}

import pandas as pd
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Save the train and test datasets to new CSV files
train_data.to_csv('train.csv', index=False)
test_data.to_csv('test.csv', index=False)

print("Data has been split and saved to train.csv and test.csv.")


Data has been split and saved to train.csv and test.csv.


In [6]:
!pip -q install pyarrow datasets

##### 3.1 Split and Load the dataset into Pytorch Dataloader

In [7]:
from datasets import load_dataset
data_files = {"train": "/content/train.csv", "test": "/content/test.csv"}
dataset = load_dataset("csv", data_files=data_files)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [8]:
dataset = dataset["train"]
dataset

Dataset({
    features: ['question', 'topic'],
    num_rows: 1274
})

In [9]:
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/1274 [00:00<?, ? examples/s]

##### 3.2 Training model

In [10]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs=1,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/1274 [00:00<?, ? examples/s]

In [11]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,274 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 159
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,4.035400
2,3.588500
3,3.762400
4,3.114700
5,2.532100
6,1.880000
7,1.416000
8,1.286600
9,1.182400
10,1.146200


#### 4. Inference on single `question`

In [12]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        dataset["question"][2],#question
        "", # topic for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 20, use_cache = True)
a=tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [13]:
import re
pattern = r'### topic:\n(.+)<'

# Search for the pattern in the text
match = re.search(pattern, a[0])

if match:
    topic = match.group(1)
    print(f"{topic}")
    if(topic==dataset["topic"][2]):
      print("Correct")
    else:
      print("Incorrect")
else:
    print("No topic found.")

Finance
Correct


##### 4.1 Inferene on test dataset

In [14]:
from datasets import load_dataset
data_files = {"train": "/content/train.csv", "test": "/content/test.csv"}
dataset = load_dataset("csv", data_files=data_files)

In [15]:
dataset_test= dataset["test"]

In [16]:
dataset_test['question'][0]

'Guess the name of the girl'

In [17]:
len(dataset_test['question'])

319

In [18]:
dataset_test

Dataset({
    features: ['question', 'topic'],
    num_rows: 319
})

In [19]:
c=0
y_true_words = []
y_pred_words = []
for i in range(len(dataset_test['question'])):

  FastLanguageModel.for_inference(model)
  inputs = tokenizer(
  [
      alpaca_prompt.format(
          dataset_test['question'][i],#question
          "", # topic - generation!
      )
  ], return_tensors = "pt").to("cuda")
  outputs = model.generate(**inputs, max_new_tokens = 20, use_cache = True)
  a=tokenizer.batch_decode(outputs)
  import re
  pattern = r'### topic:\n(.+)<'

  # Search for the pattern in the text
  match = re.search(pattern, a[0])

  if match:
      topic = match.group(1)
      y_pred_words.append(topic)
      y_true_words.append(dataset_test["topic"][i])
      #print(f"{topic}")
      if(topic==dataset_test["topic"][i]):
        #print("Correct")
        c+=1
      else:
        print("Incorrect")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Incorrect


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Incorrect


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Incorrect


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Incorrect


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Incorrect


In [20]:
c

240

#### 5. Computing Metrics

In [21]:
print(f"Accuracy is {c/len(dataset_test['question'])}")

Accuracy is 0.7523510971786834


In [25]:
from collections import defaultdict

classes = ['Dating and Relationships', 'Random', 'Movies and TV', 'Music', 'Technology and Internet', 'Philosophy and Religion', 'technology and Internet', 'Law and Order', 'Health and Wellness', 'Food and Drinks', 'Fashion and Makeup', 'health and Wellness', 'Gaming', 'Sports', 'fashion and Makeup', 'sports', 'Anime', 'anime', 'Finance', 'Travel and Geography', 'Pets and Animals', 'gaming', 'random', 'music','History and Geography']

# Convert words to class indices
def word_to_class_index(word):
    return classes.index(word)

y_true = [word_to_class_index(word) for word in y_true_words]
y_pred = [word_to_class_index(word) for word in y_pred_words]

# Initialize dictionaries to store counts for each class
class_counts = defaultdict(lambda: {'TP': 0, 'FP': 0, 'FN': 0})

# Calculate TP, FP, FN for each class
for true, pred in zip(y_true, y_pred):
    if true == pred:
        class_counts[true]['TP'] += 1
    else:
        class_counts[true]['FN'] += 1
        class_counts[pred]['FP'] += 1

# Calculate Precision and Recall for each class
class_metrics = {}
for cls in classes:
    TP = class_counts[classes.index(cls)]['TP']
    FP = class_counts[classes.index(cls)]['FP']
    FN = class_counts[classes.index(cls)]['FN']

    precision = TP / (TP + FP) if (TP + FP) != 0 else 0
    recall = TP / (TP + FN) if (TP + FN) != 0 else 0

    class_metrics[cls] = {'Precision': precision, 'Recall': recall}

# Print results
for cls, metrics in class_metrics.items():
    print(f'Class: {cls}')
    print(f'Precision: {metrics["Precision"]:.2f}')
    print(f'Recall: {metrics["Recall"]:.2f}')
    print()


Class: Dating and Relationships
Precision: 0.33
Recall: 0.40

Class: Random
Precision: 0.74
Recall: 0.74

Class: Movies and TV
Precision: 0.75
Recall: 0.94

Class: Music
Precision: 0.82
Recall: 0.90

Class: Technology and Internet
Precision: 0.42
Recall: 0.45

Class: Philosophy and Religion
Precision: 0.33
Recall: 0.14

Class: technology and Internet
Precision: 0.00
Recall: 0.00

Class: Law and Order
Precision: 0.68
Recall: 1.00

Class: Health and Wellness
Precision: 0.33
Recall: 0.50

Class: Food and Drinks
Precision: 0.92
Recall: 0.85

Class: Fashion and Makeup
Precision: 0.88
Recall: 0.64

Class: health and Wellness
Precision: 0.00
Recall: 0.00

Class: Gaming
Precision: 0.94
Recall: 0.94

Class: Sports
Precision: 0.90
Recall: 0.95

Class: fashion and Makeup
Precision: 0.00
Recall: 0.00

Class: sports
Precision: 0.00
Recall: 0.00

Class: Anime
Precision: 0.82
Recall: 0.88

Class: anime
Precision: 0.00
Recall: 0.00

Class: Finance
Precision: 1.00
Recall: 0.88

Class: Travel and Geogra